# Workflow vs Agent: Understanding the Key Differences

This notebook demonstrates the difference between **Workflows** (traditional deterministic approach) and **Agents** (adaptive LangGraph approach) using a joke generation example.

## Key Concepts:
- **Workflow**: Fixed, predictable sequence of operations
- **Agent**: Dynamic, adaptive decision-making with self-correction capabilities

## 1. Setup and Dependencies

In [1]:
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, END

# Load environment
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-latest", anthropic_api_key=api_key)

print("✅ Dependencies loaded successfully")

✅ Dependencies loaded successfully


## 2. Shared Components

Both approaches will use the same state definition and core functions:

In [2]:
# Shared state definition
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str
    error_message: str
    attempt_count: int

# Shared core functions
def generate_joke(state: State):
    """Generate initial joke using LLM"""
    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}

def improve_joke(state: State):
    """Improve joke by adding wordplay"""
    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    return {"improved_joke": msg.content}

def polish_joke(state: State):
    """Add final polish with surprising twist"""
    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    return {"final_joke": msg.content}

def check_punchline(state: State):
    """Simple quality check for punchline"""
    return "Pass" if ("?" in state["joke"] or "!" in state["joke"]) else "Fail"

print("✅ Shared components defined")

✅ Shared components defined


## 3. Workflow Approach 🔄

**Characteristics:**
- ✅ Deterministic execution
- ✅ Predictable flow
- ✅ Simple to debug
- ❌ Rigid - fails fast
- ❌ No self-correction

**Flow:** Generate → Quality Gate → (Pass: Improve → Polish) | (Fail: Stop)

In [3]:
def workflow_joke_generator(topic: str) -> State:
    """
    WORKFLOW: Traditional deterministic approach
    Fixed sequence with simple conditional logic
    """
    
    state = State(
        topic=topic, 
        joke="", 
        improved_joke="", 
        final_joke="", 
        error_message="",
        attempt_count=1
    )
    
    try:
        # Step 1: Generate initial joke
        print("🔄 WORKFLOW Step 1: Generating initial joke...")
        result = generate_joke(state)
        state.update(result)
        print(f"   Generated: {state['joke'][:50]}...")
        
        # Step 2: Quality gate (hardcoded business logic)
        print("🔄 WORKFLOW Step 2: Checking quality gate...")
        gate_result = check_punchline(state)
        
        if gate_result == "Fail":
            state["error_message"] = "❌ Joke failed quality gate - no punchline detected!"
            print(f"   {state['error_message']}")
            return state
        
        print("   ✅ Quality gate passed")
        
        # Step 3: Improve joke (only if gate passed)
        print("🔄 WORKFLOW Step 3: Improving joke...")
        result = improve_joke(state)
        state.update(result)
        
        # Step 4: Polish joke
        print("🔄 WORKFLOW Step 4: Polishing joke...")
        result = polish_joke(state)
        state.update(result)
        
        print("✅ WORKFLOW: Complete!")
        
    except Exception as e:
        state["error_message"] = f"❌ Workflow failed: {str(e)}"
        print(state["error_message"])
    
    return state

print("✅ Workflow implementation ready")

✅ Workflow implementation ready


## 4. Agent Approach 🤖

**Characteristics:**
- ✅ Adaptive behavior
- ✅ Self-correction
- ✅ Complex decision trees
- ✅ Error recovery
- ❌ More complex to debug
- ❌ Potentially slower

**Flow:** Generate → Smart Gate → (Multiple paths: Improve, Polish, Regenerate, Loop)

In [4]:
# Enhanced agent functions with adaptive behavior

def agent_generate_joke(state: State):
    """Agent version with retry logic and attempt tracking"""
    try:
        result = generate_joke(state)
        print(f"🤖 AGENT: Generated joke (attempt {state.get('attempt_count', 1)})")
        print(f"   Content: {result['joke'][:60]}...")
        return result
    except Exception as e:
        print(f"🤖 AGENT: Error generating joke, using fallback: {e}")
        return {"joke": "Why did the developer go broke? Because they used up all their cache!"}

def agent_improve_joke(state: State):
    """Agent version with quality enhancement logic"""
    result = improve_joke(state)
    print("🤖 AGENT: Enhanced joke with wordplay")
    
    # Agent makes decisions based on intermediate results
    if len(result["improved_joke"]) < 50:
        print("🤖 AGENT: Joke too short, adding more content...")
        msg = llm.invoke(f"Make this joke longer and more elaborate: {result['improved_joke']}")
        result["improved_joke"] = msg.content
        print("   ✅ Enhanced with additional content")
        
    return result

def agent_polish_joke(state: State):
    """Agent version with final optimization"""
    result = polish_joke(state)
    print("🤖 AGENT: Applied final polish and twist")
    return result

def agent_regenerate_joke(state: State):
    """Agent can regenerate with improved prompting"""
    attempt = state.get('attempt_count', 1) + 1
    print(f"🤖 AGENT: Regenerating joke (attempt {attempt}) with enhanced prompt...")
    
    # More specific prompt based on what failed
    enhanced_prompt = f"""Write a funny joke about {state['topic']} that includes:
    - A clear setup and punchline
    - Either a question (?) or exclamation (!)
    - At least 10 words
    Make it clever and engaging."""
    
    msg = llm.invoke(enhanced_prompt)
    return {"joke": msg.content, "attempt_count": attempt}

print("✅ Agent functions defined")

✅ Agent functions defined


In [5]:
def agent_quality_gate(state: State):
    """Smart routing logic with multiple criteria"""
    joke = state["joke"]
    attempt = state.get('attempt_count', 1)
    
    # Multiple quality checks
    has_punchline = "?" in joke or "!" in joke
    has_setup = len(joke.split()) > 5
    has_topic = state["topic"].lower() in joke.lower()
    is_reasonable_length = 10 < len(joke) < 500
    
    print(f"🤖 AGENT: Quality assessment (attempt {attempt}):")
    print(f"   Has punchline: {has_punchline}")
    print(f"   Has setup: {has_setup}")
    print(f"   Mentions topic: {has_topic}")
    print(f"   Reasonable length: {is_reasonable_length}")
    
    # Adaptive routing logic
    if all([has_punchline, has_setup, has_topic, is_reasonable_length]):
        print("   🎯 High quality - proceeding to improvement")
        return "improve"
    elif has_punchline and has_setup:
        print("   ⚡ Basic quality met - skipping to polish")
        return "polish"
    elif attempt >= 3:
        print("   🚫 Max attempts reached - proceeding with current joke")
        return "polish"
    else:
        print("   🔄 Quality issues detected - regenerating")
        return "regenerate"

print("✅ Agent routing logic defined")

✅ Agent routing logic defined


In [6]:
def create_agent_joke_generator():
    """Build the agent graph with adaptive routing"""
    
    # Build the agent graph
    workflow = StateGraph(State)
    
    # Add nodes
    workflow.add_node("generate", agent_generate_joke)
    workflow.add_node("regenerate", agent_regenerate_joke)
    workflow.add_node("improve", agent_improve_joke)
    workflow.add_node("polish", agent_polish_joke)
    
    # Set entry point
    workflow.set_entry_point("generate")
    
    # Add conditional edges (agent decision points)
    workflow.add_conditional_edges(
        "generate",
        agent_quality_gate,
        {
            "improve": "improve",
            "polish": "polish", 
            "regenerate": "regenerate"
        }
    )
    
    # Regenerate can loop back through quality gate
    workflow.add_conditional_edges(
        "regenerate",
        agent_quality_gate,
        {
            "improve": "improve",
            "polish": "polish",
            "regenerate": "regenerate"  # Potential loop with attempt limit
        }
    )
    
    # Linear flow after quality gate passes
    workflow.add_edge("improve", "polish")
    workflow.add_edge("polish", END)
    
    return workflow.compile()

# Create the agent
agent = create_agent_joke_generator()
print("✅ Agent graph created and compiled")

✅ Agent graph created and compiled


## 5. Live Comparison Demo 🚀

Let's run both approaches side-by-side to see the differences in action:

In [7]:
# Test the WORKFLOW approach
print("="*80)
print("🔄 WORKFLOW APPROACH - DETERMINISTIC EXECUTION")
print("="*80)

topic = "programming"
workflow_result = workflow_joke_generator(topic)

print("\n📊 WORKFLOW RESULTS:")
print("-" * 40)
if workflow_result.get("error_message"):
    print(f"❌ Error: {workflow_result['error_message']}")
    if workflow_result.get('joke'):
        print(f"🎭 Initial joke: {workflow_result['joke']}")
else:
    print(f"✅ Success!")
    print(f"🎭 Final result: {workflow_result.get('final_joke', 'No final joke')}")

workflow_result

🔄 WORKFLOW APPROACH - DETERMINISTIC EXECUTION
🔄 WORKFLOW Step 1: Generating initial joke...
   Generated: Here's a programming joke:

Why do programmers pre...
🔄 WORKFLOW Step 2: Checking quality gate...
   ✅ Quality gate passed
🔄 WORKFLOW Step 3: Improving joke...
🔄 WORKFLOW Step 4: Polishing joke...
✅ WORKFLOW: Complete!

📊 WORKFLOW RESULTS:
----------------------------------------
✅ Success!
🎭 Final result: Here's a twist that adds an unexpected layer to the joke:

Why do programmers prefer dark mode?
Because light attracts bugs... but plot twist: they actually work at a butterfly sanctuary and need to protect their rare specimen collection! 🦋

This twist subverts expectations by:
1. Making you think it's about code bugs
2. Revealing it's about actual insects
3. Adding an unexpectedly wholesome reason
4. Creating a humorous image of programmers moonlighting as butterfly conservationists

Alternative twist:
Why do programmers prefer dark mode?
Because light attracts bugs... except fo

{'topic': 'programming',
 'joke': "Here's a programming joke:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! 🪲",
 'improved_joke': 'Here are a few versions with added wordplay:\n\n1. Why do programmers prefer dark mode?\nBecause light attracts bugs - they\'re trying not to get BYTE! 🪲\n\n2. Why do programmers prefer dark mode?\nBecause light attracts bugs, and they don\'t want their code to be HIGHLIGHTED! 🪲\n\n3. Why do programmers prefer dark mode?\nBecause light attracts bugs - it\'s a TERMINAL condition! 🪲\n\n4. Why do programmers prefer dark mode?\nBecause light attracts bugs, and they want to keep their code out of the SPOTLIGHT! 🪲\n\n5. Why do programmers prefer dark mode?\nBecause light attracts bugs - they don\'t want to DEBUG the brightness! 🪲\n\nThe last one is probably my favorite since it incorporates both the programming term "debug" and plays on the light/brightness theme!',
 'final_joke': "Here's a twist that adds an unexpected layer to the joke

In [8]:
# Test the AGENT approach
print("="*80)
print("🤖 AGENT APPROACH - ADAPTIVE EXECUTION")
print("="*80)

initial_state = {
    "topic": "programming", 
    "joke": "", 
    "improved_joke": "", 
    "final_joke": "", 
    "error_message": "",
    "attempt_count": 1
}

agent_result = agent.invoke(initial_state)

print("\n📊 AGENT RESULTS:")
print("-" * 40)
print(f"✅ Success!")
print(f"🎭 Final result: {agent_result.get('final_joke', 'No final joke')}")
print(f"🔄 Total attempts: {agent_result.get('attempt_count', 1)}")

agent_result

🤖 AGENT APPROACH - ADAPTIVE EXECUTION
🤖 AGENT: Generated joke (attempt 1)
   Content: Here's a programming joke:

Why do programmers prefer dark m...
🤖 AGENT: Quality assessment (attempt 1):
   Has punchline: True
   Has setup: True
   Mentions topic: True
   Reasonable length: True
   🎯 High quality - proceeding to improvement
🤖 AGENT: Enhanced joke with wordplay
🤖 AGENT: Applied final polish and twist

📊 AGENT RESULTS:
----------------------------------------
✅ Success!
🎭 Final result: Here's a twist version that subverts expectations:

Why do programmers prefer dark mode?
Because light attracts bugs - or so they thought, until they discovered their rubber duck debugger glows in the dark! 🦆✨

This twist is fun because it:
1. Sets up the familiar premise about avoiding bugs
2. Then reveals programmers accidentally created a new bug-attracting problem with their glowing rubber duck debugging tool
3. References the common programming practice of rubber duck debugging
4. Creates an absur

{'topic': 'programming',
 'joke': "Here's a programming joke:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! 🪲",
 'improved_joke': 'Here are a few wordplay-enhanced versions:\n\n1. Why do programmers prefer dark mode?\nBecause light attracts bugs - and they\'re trying to avoid de-bugging! 🪲\n\n2. Why do programmers prefer dark mode?\nBecause light attracts bugs - they don\'t want to HIGHLIGHT their problems! 🪲\n\n3. Why do programmers prefer dark mode?\nBecause light attracts bugs - and they want to keep their code out of the SPOTLIGHT! 🪲\n\n4. Why do programmers prefer dark mode?\nBecause light attracts bugs - they\'re trying to stay in their COMPILE-ment zone! 🪲\n\nThe last one is my favorite since it plays on "compliment/complement" and "compile" - all programmer-relevant terms!',
 'final_joke': "Here's a twist version that subverts expectations:\n\nWhy do programmers prefer dark mode?\nBecause light attracts bugs - or so they thought, until they discovered 

## 6. Side-by-Side Analysis 📊

Let's test both approaches with a "difficult" topic to see how they handle edge cases:

In [9]:
def compare_approaches(topic: str):
    """Run both approaches and compare results"""
    
    print(f"🎯 Testing topic: '{topic}'")
    print("="*80)
    
    # Workflow test
    print("🔄 WORKFLOW:")
    workflow_start = __import__('time').time()
    workflow_result = workflow_joke_generator(topic)
    workflow_time = __import__('time').time() - workflow_start
    
    workflow_success = not bool(workflow_result.get("error_message"))
    
    # Agent test  
    print("\n🤖 AGENT:")
    agent_start = __import__('time').time()
    agent_result = agent.invoke({
        "topic": topic, "joke": "", "improved_joke": "", 
        "final_joke": "", "error_message": "", "attempt_count": 1
    })
    agent_time = __import__('time').time() - agent_start
    
    # Comparison summary
    print("\n" + "="*80)
    print("📊 COMPARISON SUMMARY")
    print("="*80)
    
    print(f"⏱️  Execution Time:")
    print(f"   Workflow: {workflow_time:.2f}s")
    print(f"   Agent:    {agent_time:.2f}s")
    
    print(f"\n✅ Success Rate:")
    print(f"   Workflow: {'✅ Success' if workflow_success else '❌ Failed'}")
    print(f"   Agent:    ✅ Success (adaptive recovery)")
    
    print(f"\n🔄 Adaptability:")
    print(f"   Workflow: Fixed path, fails at quality gate")
    print(f"   Agent:    {agent_result.get('attempt_count', 1)} attempts, self-correcting")
    
    return workflow_result, agent_result

# Test with a potentially challenging topic
workflow_res, agent_res = compare_approaches("quantum computing")

🎯 Testing topic: 'quantum computing'
🔄 WORKFLOW:
🔄 WORKFLOW Step 1: Generating initial joke...
   Generated: Here's a quantum computing joke:

Why did the quan...
🔄 WORKFLOW Step 2: Checking quality gate...
   ✅ Quality gate passed
🔄 WORKFLOW Step 3: Improving joke...
🔄 WORKFLOW Step 4: Polishing joke...
✅ WORKFLOW: Complete!

🤖 AGENT:
🤖 AGENT: Generated joke (attempt 1)
   Content: Here's a quantum computing joke:

Why don't quantum computer...
🤖 AGENT: Quality assessment (attempt 1):
   Has punchline: True
   Has setup: True
   Mentions topic: True
   Reasonable length: True
   🎯 High quality - proceeding to improvement
🤖 AGENT: Enhanced joke with wordplay
🤖 AGENT: Applied final polish and twist

📊 COMPARISON SUMMARY
⏱️  Execution Time:
   Workflow: 11.98s
   Agent:    10.72s

✅ Success Rate:
   Workflow: ✅ Success
   Agent:    ✅ Success (adaptive recovery)

🔄 Adaptability:
   Workflow: Fixed path, fails at quality gate
   Agent:    1 attempts, self-correcting


## 7. Key Differences Summary 📋

| Aspect | Workflow 🔄 | Agent 🤖 |
|--------|-------------|----------|
| **Execution** | Deterministic, linear | Adaptive, non-linear |
| **Decision Making** | Hardcoded conditional logic | Dynamic routing based on state |
| **Error Handling** | Fail-fast, no recovery | Self-correction and retry |
| **Complexity** | Simple, predictable | Complex, intelligent |
| **Performance** | Fast, efficient | Slower, more thorough |
| **Debugging** | Easy to trace | Harder to predict |
| **Maintenance** | Simple updates | Complex state management |
| **Use Cases** | Stable, well-defined processes | Dynamic, evolving requirements |

## 8. When to Use Each Approach 🤔

### Use **Workflows** when:
- ✅ Requirements are well-defined and stable
- ✅ Simple, linear processing is sufficient  
- ✅ Predictability and performance are critical
- ✅ Easy debugging and maintenance are important
- ✅ Compliance and auditability are required

**Examples:** Data ETL pipelines, report generation, simple approval processes

### Use **Agents** when:
- ✅ Requirements are complex or evolving
- ✅ Need adaptive behavior and error recovery
- ✅ Multiple decision paths are required
- ✅ Self-optimization adds significant value
- ✅ Handling unpredictable inputs

**Examples:** Content generation, customer service, research tasks, complex problem-solving

## 9. Cost and Performance Analysis 💰

In [10]:
def estimate_costs(workflow_result, agent_result):
    """
    Rough cost estimation based on LLM calls
    Note: This is a simplified example - real costs depend on token usage
    """
    
    # Typical workflow: 3 LLM calls (generate, improve, polish)
    workflow_calls = 3 if not workflow_result.get('error_message') else 1
    
    # Agent calls depend on routing decisions
    agent_attempts = agent_result.get('attempt_count', 1)
    # Base calls: generate + routing, then improve + polish (or just polish)
    agent_calls = agent_attempts + (3 if 'improved_joke' in agent_result else 2)
    
    # Rough cost per call (example - actual costs vary)
    cost_per_call = 0.003  # ~$0.003 per call for simple jokes
    
    workflow_cost = workflow_calls * cost_per_call
    agent_cost = agent_calls * cost_per_call
    
    print("💰 ESTIMATED COSTS:")
    print("-" * 40)
    print(f"Workflow: {workflow_calls} calls × ${cost_per_call:.3f} = ${workflow_cost:.3f}")
    print(f"Agent:    {agent_calls} calls × ${cost_per_call:.3f} = ${agent_cost:.3f}")
    print(f"\nCost difference: ${abs(agent_cost - workflow_cost):.3f} ({((agent_cost/workflow_cost - 1)*100):+.1f}%)")
    
    print("\n📈 TRADE-OFFS:")
    print("-" * 40)
    if workflow_result.get('error_message'):
        print("• Workflow failed - Agent provided working solution despite higher cost")
        print("• Agent's self-correction justified the additional expense")
    else:
        print("• Both succeeded - evaluate if agent's adaptability worth extra cost")
        print("• Consider failure rates in production when choosing approach")

# Run cost analysis
estimate_costs(workflow_res, agent_res)

💰 ESTIMATED COSTS:
----------------------------------------
Workflow: 3 calls × $0.003 = $0.009
Agent:    4 calls × $0.003 = $0.012

Cost difference: $0.003 (+33.3%)

📈 TRADE-OFFS:
----------------------------------------
• Both succeeded - evaluate if agent's adaptability worth extra cost
• Consider failure rates in production when choosing approach


## 10. Conclusion and Recommendations 🎯

### For Your SWE Team:

1. **Start Simple**: Begin with workflows for well-understood processes
2. **Evolve to Agents**: When requirements become complex or unpredictable
3. **Hybrid Approach**: Use workflows for stable components, agents for dynamic parts
4. **Monitor Costs**: Agent flexibility comes at computational cost
5. **Testing Strategy**: Agents require more sophisticated testing due to non-deterministic behavior

### Architecture Decision Framework:

```
Requirements Stable + Simple Logic → Workflow
Requirements Dynamic + Complex Logic → Agent  
Cost Sensitive + Predictable → Workflow
Quality Critical + Adaptive → Agent
```

The key is matching the tool to the problem complexity and business requirements! 🚀